In [1]:
import os
import pandas as pd
import numpy as np
import requests
import time
pd.set_option('display.max_columns', None)

In [2]:
# nombre del archivo con la información a geocodificar y su terminación (CSV)
archivo = "directorio_anuies.csv"

In [3]:
# Lectura
df = pd.read_csv(os.path.join('..','datos', archivo ))
print(df.shape)
df.head()

(244, 21)


,Unnamed: 0,institucion,fecha_creacion,niveles_acad,niv_acad_1,niv_acad_2,niv_acad_3,niv_acad_4,niv_acad_5,niv_acad_6,direccion_com,direc_1,direc_2,direc_3,direc_4,direc_5,direc_6,telefono,fax,web,email
0,0,Instituto Tecnológico de Aguascalientes (ITA),18/09/1967,LicenciaturaPosgradoMaestríaDoctorado,Licenciatura,Posgrado,Maestría,Doctorado,NaN,NaN,"Av. Adolfo López Mateos N° 1801 Ote.\r, Col. F...","Av. Adolfo López Mateos N° 1801 Ote.\r, Col. F...",Aguascalientes,NaN,Aguascalientes,Aguascalientes,20256,(449),(449),https://aguascalientes.tecnm.mx/,"dir_aguascalientes@tecnm.mx, direccion@aguasca..."
1,1,Universidad Autónoma de Aguascalientes (UAA),24/02/1974,BachilleratoLicenciaturaPosgradoEspecialidadMa...,Bachillerato,Licenciatura,Posgrado,Especialidad,Maestría,Doctorado,"Blvd. Av. Universidad N° 940\r, Edificio 1, Ci...","Blvd. Av. Universidad N° 940\r, Edificio 1, Ci...",Aguascalientes,NaN,Aguascalientes,Aguascalientes,20100,"(449) 910 74 10, 910 74 08",(449),https://www.uaa.mx/portal/,yesenia.pinzon@edu.uaa.mx
2,2,Universidad Politécnica de Aguascalientes (UPA),02/08/2002,LicenciaturaPosgradoMaestría,Licenciatura,Posgrado,Maestría,NaN,NaN,NaN,"Calle Paseo San Gerardo N° 207 \r, Fracc. San ...","Calle Paseo San Gerardo N° 207 \r, Fracc. San ...",Aguascalientes,NaN,Aguascalientes,Aguascalientes,20342,(449),(449),https://www.upa.edu.mx/,otto.granados@upa.edu.mx
3,3,Universidad Tecnológica de Aguascalientes (UTAgs),11/08/1991,Técnico SuperiorLicenciatura,Técnico Superior,Licenciatura,NaN,NaN,NaN,NaN,"Blvd. Juan Pablo II N° 1302\r, Fracc. Ex-Hacie...","Blvd. Juan Pablo II N° 1302\r, Fracc. Ex-Hacie...",Aguascalientes,NaN,Aguascalientes,Aguascalientes,20200,(449),(449),https://www.utags.edu.mx/,armando.lopez@utags.edu.mx; luis.cardenas@utag...
4,4,Universidad Tecnológica de Calvillo (UTCalvillo),08/07/2013,Técnico SuperiorLicenciatura,Técnico Superior,Licenciatura,NaN,NaN,NaN,NaN,"Carr. al Tepetate 102\r, Col. El SalitreAguasc...","Carr. al Tepetate 102\r, Col. El Salitre",Aguascalientes,NaN,Calvillo,Aguascalientes,20860,(495),(495),http://www.utcalvillo.edu.mx/,eduardo.blas@utcalvillo.edu.mx


In [4]:
# Concatenar columnas
# df['concatenada'] = df[['col1', 'col2', 'col3']].astype(str).agg(lambda x: ', '.join(x.dropna()), axis=1)
df.insert(10, 'direccion', df[['direc_1', 'direc_2', 'direc_4', 'direc_5', 'direc_6']].astype(str).agg(lambda x: ', '.join(x.dropna()), axis=1))
df['direccion'] = df['direccion'].str.replace(r"[\r]", "", regex=True)
df.head()

,Unnamed: 0,institucion,fecha_creacion,niveles_acad,niv_acad_1,niv_acad_2,niv_acad_3,niv_acad_4,niv_acad_5,niv_acad_6,direccion,direccion_com,direc_1,direc_2,direc_3,direc_4,direc_5,direc_6,telefono,fax,web,email
0,0,Instituto Tecnológico de Aguascalientes (ITA),18/09/1967,LicenciaturaPosgradoMaestríaDoctorado,Licenciatura,Posgrado,Maestría,Doctorado,NaN,NaN,"Av. Adolfo López Mateos N° 1801 Ote., Col. Fra...","Av. Adolfo López Mateos N° 1801 Ote.\r, Col. F...","Av. Adolfo López Mateos N° 1801 Ote.\r, Col. F...",Aguascalientes,NaN,Aguascalientes,Aguascalientes,20256,(449),(449),https://aguascalientes.tecnm.mx/,"dir_aguascalientes@tecnm.mx, direccion@aguasca..."
1,1,Universidad Autónoma de Aguascalientes (UAA),24/02/1974,BachilleratoLicenciaturaPosgradoEspecialidadMa...,Bachillerato,Licenciatura,Posgrado,Especialidad,Maestría,Doctorado,"Blvd. Av. Universidad N° 940, Edificio 1, Ciud...","Blvd. Av. Universidad N° 940\r, Edificio 1, Ci...","Blvd. Av. Universidad N° 940\r, Edificio 1, Ci...",Aguascalientes,NaN,Aguascalientes,Aguascalientes,20100,"(449) 910 74 10, 910 74 08",(449),https://www.uaa.mx/portal/,yesenia.pinzon@edu.uaa.mx
2,2,Universidad Politécnica de Aguascalientes (UPA),02/08/2002,LicenciaturaPosgradoMaestría,Licenciatura,Posgrado,Maestría,NaN,NaN,NaN,"Calle Paseo San Gerardo N° 207 , Fracc. San Ge...","Calle Paseo San Gerardo N° 207 \r, Fracc. San ...","Calle Paseo San Gerardo N° 207 \r, Fracc. San ...",Aguascalientes,NaN,Aguascalientes,Aguascalientes,20342,(449),(449),https://www.upa.edu.mx/,otto.granados@upa.edu.mx
3,3,Universidad Tecnológica de Aguascalientes (UTAgs),11/08/1991,Técnico SuperiorLicenciatura,Técnico Superior,Licenciatura,NaN,NaN,NaN,NaN,"Blvd. Juan Pablo II N° 1302, Fracc. Ex-Haciend...","Blvd. Juan Pablo II N° 1302\r, Fracc. Ex-Hacie...","Blvd. Juan Pablo II N° 1302\r, Fracc. Ex-Hacie...",Aguascalientes,NaN,Aguascalientes,Aguascalientes,20200,(449),(449),https://www.utags.edu.mx/,armando.lopez@utags.edu.mx; luis.cardenas@utag...
4,4,Universidad Tecnológica de Calvillo (UTCalvillo),08/07/2013,Técnico SuperiorLicenciatura,Técnico Superior,Licenciatura,NaN,NaN,NaN,NaN,"Carr. al Tepetate 102, Col. El Salitre, Aguasc...","Carr. al Tepetate 102\r, Col. El SalitreAguasc...","Carr. al Tepetate 102\r, Col. El Salitre",Aguascalientes,NaN,Calvillo,Aguascalientes,20860,(495),(495),http://www.utcalvillo.edu.mx/,eduardo.blas@utcalvillo.edu.mx


In [5]:
# Configuración de la API de Nominatim
NOMINATIM_URL = "https://nominatim.openstreetmap.org/search"
HEADERS = {"User-Agent": "MiScriptGeocodificador/1.0"}

In [6]:
# Función para buscar un lugar y obtener sus coordenadas
def buscar_lugar(lugar):
    try:
        params = {"q": lugar, "format": "json", "limit": 1}  # Busca solo el mejor resultado
        response = requests.get(NOMINATIM_URL, params=params, headers=HEADERS, timeout=10)
        
        if response.status_code == 200:
            data = response.json()
            if data:
                return data[0]["lat"], data[0]["lon"]  # Devuelve latitud y longitud
    except Exception as e:
        print(f"Error con '{lugar}': {e}")

    return None, None  # Si hay error, devuelve valores vacíos

In [7]:
# Aplicar la función al DataFrame
df["latitud"], df["longitud"] = zip(*df['direccion'].apply(buscar_lugar))

In [8]:
df.head()

,Unnamed: 0,institucion,fecha_creacion,niveles_acad,niv_acad_1,niv_acad_2,niv_acad_3,niv_acad_4,niv_acad_5,niv_acad_6,direccion,direccion_com,direc_1,direc_2,direc_3,direc_4,direc_5,direc_6,telefono,fax,web,email,latitud,longitud
0,0,Instituto Tecnológico de Aguascalientes (ITA),18/09/1967,LicenciaturaPosgradoMaestríaDoctorado,Licenciatura,Posgrado,Maestría,Doctorado,NaN,NaN,"Av. Adolfo López Mateos N° 1801 Ote., Col. Fra...","Av. Adolfo López Mateos N° 1801 Ote.\r, Col. F...","Av. Adolfo López Mateos N° 1801 Ote.\r, Col. F...",Aguascalientes,NaN,Aguascalientes,Aguascalientes,20256,(449),(449),https://aguascalientes.tecnm.mx/,"dir_aguascalientes@tecnm.mx, direccion@aguasca...",None,None
1,1,Universidad Autónoma de Aguascalientes (UAA),24/02/1974,BachilleratoLicenciaturaPosgradoEspecialidadMa...,Bachillerato,Licenciatura,Posgrado,Especialidad,Maestría,Doctorado,"Blvd. Av. Universidad N° 940, Edificio 1, Ciud...","Blvd. Av. Universidad N° 940\r, Edificio 1, Ci...","Blvd. Av. Universidad N° 940\r, Edificio 1, Ci...",Aguascalientes,NaN,Aguascalientes,Aguascalientes,20100,"(449) 910 74 10, 910 74 08",(449),https://www.uaa.mx/portal/,yesenia.pinzon@edu.uaa.mx,None,None
2,2,Universidad Politécnica de Aguascalientes (UPA),02/08/2002,LicenciaturaPosgradoMaestría,Licenciatura,Posgrado,Maestría,NaN,NaN,NaN,"Calle Paseo San Gerardo N° 207 , Fracc. San Ge...","Calle Paseo San Gerardo N° 207 \r, Fracc. San ...","Calle Paseo San Gerardo N° 207 \r, Fracc. San ...",Aguascalientes,NaN,Aguascalientes,Aguascalientes,20342,(449),(449),https://www.upa.edu.mx/,otto.granados@upa.edu.mx,None,None
3,3,Universidad Tecnológica de Aguascalientes (UTAgs),11/08/1991,Técnico SuperiorLicenciatura,Técnico Superior,Licenciatura,NaN,NaN,NaN,NaN,"Blvd. Juan Pablo II N° 1302, Fracc. Ex-Haciend...","Blvd. Juan Pablo II N° 1302\r, Fracc. Ex-Hacie...","Blvd. Juan Pablo II N° 1302\r, Fracc. Ex-Hacie...",Aguascalientes,NaN,Aguascalientes,Aguascalientes,20200,(449),(449),https://www.utags.edu.mx/,armando.lopez@utags.edu.mx; luis.cardenas@utag...,None,None
4,4,Universidad Tecnológica de Calvillo (UTCalvillo),08/07/2013,Técnico SuperiorLicenciatura,Técnico Superior,Licenciatura,NaN,NaN,NaN,NaN,"Carr. al Tepetate 102, Col. El Salitre, Aguasc...","Carr. al Tepetate 102\r, Col. El SalitreAguasc...","Carr. al Tepetate 102\r, Col. El Salitre",Aguascalientes,NaN,Calvillo,Aguascalientes,20860,(495),(495),http://www.utcalvillo.edu.mx/,eduardo.blas@utcalvillo.edu.mx,None,None


In [9]:
df.to_csv(os.path.join('..', 'datos', 'directorio_anuies_01.csv'))